In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Britney Spears,Mornin',Early mornin' \r\nGreat \r\nI was out real l...
1,Peter Gabriel,Growing Up,Folded in your fleshy purse \r\nI am floating...
2,George Harrison,Dream Away,Oh Ry In Eye Ay - Oh Ry In Eye Ay \r\nOh Ry I...
3,The Temptations,(You Can) Depend On Me,Hmmmmmm... \r\nYou... \r\nNeed a love like m...
4,Beautiful South,Here It Is Again,Here it is again it's so so mad \r\nTurning y...
5,The Script,Hall Of Fame,"Yeah, you can be the greatest \r\nYou can be ..."
6,Tori Amos,Carbon,carbon made found her at the \r\nEnd of a cha...
7,Natalie Imbruglia,Identify,"Your eyes, they send me \r\nTo eternity \r\n..."
8,Harry Belafonte,Go Down Emanuel Road,"Go down Emanuel Road, man and boy \r\nGot to ..."
9,Louis Jordan,Lemonade,Yes I'm looking for a chick \r\nThat only dri...


In [9]:
df['text'][0]

"Early mornin'  \r\nGreat  \r\nI was out real late last night  \r\nGot a little messy  \r\nEarly morning  \r\nCan't be like that anymore  \r\nNo  \r\n  \r\nI was shaking my ass in the streets alone and  \r\nJust walked in and it's early mornin'  \r\nBump, bump till the break of dawn  \r\nAnd it don't stop till the early mornin'  \r\nPassed out on the couch and yawnin'  \r\nJust walked in and it's early mornin'  \r\nBump, bump till the break of dawn  \r\nAnd it don't stop till the early mornin'  \r\n  \r\nMet a tall dude, kinda dark hair  \r\nWhen he walked up somebody grabbed him  \r\nBut I liked him told him come here  \r\nKinda cool  \r\nBaby, we can make plans  \r\nWhere ya live  \r\nDo ya mama live there  \r\nWe can hook up at the hotel  \r\nHands down so I told him, let's go  \r\nBut have a next guess  \r\nBut you don't wanna know  \r\n  \r\nI was shaking my ass in the streets alone and  \r\nJust walked in and it's early mornin'  \r\nBump, bump till the break of dawn  \r\nAnd it d

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk

# Download the necessary NLTK data files for tokenization
nltk.download('punkt')

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

[nltk_data] Downloading package punkt to /Users/hina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.02302463, 0.00259548, ..., 0.01847379, 0.02286927,
       0.02758081])

In [18]:
df[df['song'] == 'Crying Over You']

,artist,song,text
3433,UB40,Crying Over You,cri over you in the morn cri over you in the e...


In [19]:
def recommendation(song_df):
    # Filter DataFrame based on song name
    filtered_df = df[df['song'] == song_df]
    
    # Check if any rows match the query
    if filtered_df.empty:
        print(f"No recommendations found for the song '{song_df}'.")
        return
    
    # If there are matching rows, proceed with recommendation logic
    idx = filtered_df.index[0]
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    songs = []
    # Rest of your recommendation logic...

    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [20]:
recommendation('Crying Over You')

['Crying',
 "I'm Gonna Cry (Cry Baby)",
 "It's The Falling In Love",
 'Cry Baby',
 'I Cry',
 "It's All Over But The Crying",
 'Cry A While',
 'Johnny Was',
 'Cry Me A River',
 'Cry Cry Cry',
 'Fly Away',
 'I Can Cry Again',
 'Almost Hear You Sigh',
 'Phoenix',
 'Speak To Me Lord',
 'This Is What Makes Us Girls',
 'When You Cry',
 'Do You Really Want To Hurt Me',
 'When The Time Comes',
 "Don't Pass Me By"]

In [21]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))